# Segmenting and Clustering Neighborhoods in Toronto

## Part 1

#### Getting import packages and functions

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: | 

#### Here we download the wikipedia data set using pandas

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
data = pd.read_html(url)

#### And convert it to a pandas data frame

In [ ]:
t_df = pd.DataFrame(data[0])
t_df.head()

#### Here we eliminate all rows with a "Not assigned" Burough

In [ ]:
df = t_df[t_df.Borough != "Not assigned"]
df.head(10)

#### And here we see that there are no cases of a row where the Burough is named but the Neighborhood is not

In [ ]:
df[df.Neighborhood == "Not assigned"]

#### Now we check our shape...

In [ ]:
df.shape

#### Our resulting data set has 3 columns and 103 observations!

## Part 2

#### importing geocoder to get coordinates

In [ ]:
#!conda install -c conda-forge geocoder=1.38.1 --yes
#import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### set up empty Latitude and Longitude columns in df

In [ ]:
df["Latitude"] = np.NaN
df["Longitude"] = np.NaN
df.head()

#### find the coordinates for each Postal Code

In [ ]:
#geolocator = Nominatim(user_agent="toronto_explorer")

In [ ]:
#for i in df["Postal Code"]:
    #initialize your variable to None
    #lat_lng_coords = None

    # loop until you get the coordinates
    #while(lat_lng_coords is None):
      #g = geocoder.google('{}, Toronto, Ontario'.format(i))
      #lat_lng_coords = g.latlng
        
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    
    #df.loc[df["Postal Code"]==i,"Latitude"] = latitude
    #df.loc[df["Postal Code"]==i,"Longitude"] = longitude

#### geocoder not working (never finishes running, even on a single area code) so we dowload the csv

In [ ]:
#geo_data = pd.read_csv("Geospatial_Coordinates.csv")
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c3184cd62a4c4b3a920f9b334b501109 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='idJWCKU3OzH7JulMLNlXo4IF5cXmOJOlxKqEWJYPLZis',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c3184cd62a4c4b3a920f9b334b501109.get_object(Bucket='courseracapstoneapplieddatascienc-donotdelete-pr-whqbcsawpi7ucd',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo_data = pd.read_csv(body)
geo_data.head()

In [ ]:
df.head()

#### We drop the Latitude and Longitude from our df and merge the df and geo_data by the postal code.

In [ ]:
df = df.drop(['Latitude', 'Longitude'], axis=1)
df_final = pd.merge(df, geo_data, on='Postal Code')

In [ ]:
df_final[df_final["Postal Code"]=="M5G"]

#### and we have recreate the section 2 dataframe

## Part 3

In [ ]:
import requests

In [ ]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#### Let's build a map of toronto with all of these neighborhoods in our dataset

In [ ]:
map_toronto= folium.Map(location=[latitude, longitude],zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Now we look at specifically toronto neighborhoods

In [ ]:
toronto_data = df_final[df_final['Borough'].str.contains("Toronto") == True].reset_index(drop=True)
toronto_data.head()

In [ ]:
CLIENT_ID = 'ZFPLKJUYQ4QRIQ54EXFUZIYTDJJMLA341EIHRQW3BTVWRKKU' # your Foursquare ID
CLIENT_SECRET = 'GY5KRZEPMXPQAUSYE0KR50TF4Y1MGEECFDSWG5JA4LWC55IW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Here's a function for getting the category of venue

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### This function finds the names of venues within a 500 meter radius of our neighborhood coordinates

In [ ]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we can check out the names of the venues nearby

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

#### Now we code a dataset that uses indicator variables for venue type

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Now we can see which venue types are most frequent in each neighborhood

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now we find the 10 most common venues in each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Now we cluster the nieghborhoods by similar venues

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

#### And here we map out the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Here we take a look at the first cluster of neighborhoods, by far largest cluster. The most common venues each neighborhood in this cluster has appears to be a Coffee Shop and Cafe.

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Now we look at the second cluster, which only has 3 neighborhoods in it. The most common venue for this cluster is a Park.

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### The other clusters displayed below only have a single neighborhood in each cluster

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]